In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def downsample_image(image_path, output_dir='downsampled_images', scale=0.25, noise_prob=1, noise_std=5, blur_ksize=3, artifact_prob=1, artifact_quality=80):
    """
    Downsample image with noise, blur, and artifacts
    
    Args:
        image_path: Image path
        output_dir: Directory to save the downsampled image
        scale: Downsampling factor
        noise_prob: probability of adding Gaussian noise
        noise_std: standard deviation of Gaussian noise
        blur_ksize: kernel size for Gaussian blur
        artifact_prob: probability of adding compression artifacts
        artifact_quality: quality of compression artifacts
    """
    os.makedirs(output_dir, exist_ok=True)

    # Load & downsample
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    downsampled = cv2.resize(image, (int(width * scale), int(height * scale)), interpolation=cv2.INTER_AREA)
    
    # Gaussian noise
    if np.random.uniform(0, 1) < noise_prob:
        noise = np.random.normal(0, noise_std, downsampled.shape).astype(np.uint8)
        noisy_image = cv2.add(downsampled, noise)
    else:
        noisy_image = downsampled

    # Gaussian blur
    blurred_image = cv2.GaussianBlur(noisy_image, (blur_ksize, blur_ksize), 0)

    # Compression artifacts
    if np.random.uniform(0, 1) < artifact_prob:
        _, compressed_image = cv2.imencode('.jpg', blurred_image, [cv2.IMWRITE_JPEG_QUALITY, artifact_quality])
        final_image = cv2.imdecode(compressed_image, cv2.IMREAD_COLOR)
    else:
        final_image = blurred_image

    # Save image
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, final_image)

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = image.reshape(1, -1)
    image = image / 255.0 # scale to (0, 1)
    image = image * 2 - 1 # send to (-1, 1)
    return image

In [ ]:
image_path = 'images/00000001_000.png'
orig_image = cv2.imread(image_path)
print(f'Original image shape: {orig_image.shape}')
preprocessed_image = preprocess_image(image_path)
print(f'Preprocessed image shape: {preprocessed_image.shape}')

downsample_image(image_path)
downsampled_image = cv2.imread('downsampled_images/00000001_000.png')
print(f'Downsampled image shape: {downsampled_image.shape}')

preprocessed_downsampled_image = preprocess_image('downsampled_images/00000001_000.png')
print(f'Preprocessed downsampled image shape: {preprocessed_downsampled_image.shape}')

In [ ]:
plt.imshow(orig_image, cmap='gray')

In [ ]:
plt.imshow(preprocessed_image.reshape(orig_image.shape), cmap='gray')

In [ ]:
plt.imshow(downsampled_image, cmap='gray')

In [ ]:
plt.imshow(preprocessed_downsampled_image.reshape(downsampled_image.shape), cmap='gray')